Etapa 1:Rodando o banco de dados com Docker

Caso 1: CockroachDB

Para rodar o Cockroach através do Docker usamos os seguintes comandos:

In [ ]:
sudo docker network create -d bridge roachnet

Posteriormente construimos um cluster do CockroachDB.

In [ ]:
sudo docker run -d \
--name=roach1 \
--hostname=roach1 \
--net=roachnet \
-p 26257:26257 -p 8080:8080  \
-v "${PWD}/cockroach-data/roach1:/cockroach/cockroach-data"  \
cockroachdb/cockroach:v21.2.7 start \
--insecure \
--join=roach1

Para inciar o cluster usamos:

In [ ]:
sudo docker exec -it roach1 ./cockroach sql --insecure

Caso 2: Postgresql

Para rodar a imagem do postgrtes através do docker precisamos apenas usar o comando:

In [ ]:
sudo docker run -p 5432:5432 -e POSTGRES_PASSWORD=1234 postgres

Logo após a criação dos containers dos BDs no docker, precisamos criar os databases para efetivamente usar nosso benchmark. Recomendamos o uso do DBeaver para conectar-se com os bancos e criar os databases.
Os databases são sempre recriados a cada teste, com o uso dos comandos a baixo:

In [ ]:
create database voter_base;
drop database voter_base;

Etapa 2: Configurando o XML do BenchBase 

Depois de criado o database, precisamos configurar o BenchBase. Ele pode ser configurado através dos arquivos XML que ficam no destino target/benchbase-2021-SNAPSHOT/config. A seguir temos a configuração usada para o voter. Contudo, tanto para outros Benchmarks, como para outros BDs a configuração do arquivo Xml é bastante similar.

In [ ]:
<?xml version="1.0"?>
<parameters>

    <!-- Connection details -->
    <type>POSTGRES</type>
    <driver>org.postgresql.Driver</driver>
    <url>jdbc:postgresql://localhost:5432/voter_base?sslmode=disable&amp;ApplicationName=voter&amp;reWriteBatchedInserts=true</url>
    <username>postgres</username>
    <password>1234</password>
    <isolation>TRANSACTION_SERIALIZABLE</isolation>
    <batchsize>128</batchsize>

    <scalefactor>10</scalefactor>

    <!-- The workload -->
    <terminals>10</terminals>
    <works>
        <work>
            <time>150</time>
            <rate>10000</rate>
            <weights>100</weights>
        </work>
    </works>

    <!-- Voter Procedures declaration -->
    <transactiontypes>
        <transactiontype>
            <name>Vote</name>
        </transactiontype>
    </transactiontypes>
</parameters>

Etapa 3: Execução do benchmark

Depois dos databases criados, e do arquivo de configuração pronto, podemos enfim executar o BenchBase. Para isso devemos abrir um terminal no local target/benchbase-2021-SNAPSHOT e executar o seguinte comando.

In [ ]:
java -jar benchbase.jar -b voter -c config/postgres/sample_voter_config.xml --create=true --load=true --execute=true

O BenchBase irá executar pelo tempo que foi definido no arquivo Xml, e depois do tempo decorrido era imprimir no terminal todos os resultados obtidos.